In [90]:
import numpy as np
import pandas as pd
import itertools
import random
import ast
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon, friedmanchisquare
from scipy.spatial import distance
from fastDamerauLevenshtein import damerauLevenshtein
#from opportunistic_planning import calculate_edit_distances, get_lowest_error, get_median_edit_distance, predict_sequence
from sklearn.model_selection import ParameterGrid

pd.set_option('display.max_rows', 500)

In [19]:
# read data

df = pd.read_csv('all_task_environments_2020-11-25.csv', header=0, index_col='ID')
#df_old = pd.read_csv('all_task_environments_new.csv', header=0, skiprows=[1])

In [20]:
df

,sequence,coordinates,start_coordinates,strong_k,mid_k,food_k,containment,error
ID,,,,,,,,
a1,pocgkr,"p: (0.008034,0.957082,0.6890539999999999);o: (...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,"p,o",0,0,0.723
a3,cgwpcfks,"c: (-0.525007,0.923283,0.438);g: (-0.595001,1....","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,"p,o",0,0,0.785
a5,kfsfkspwg,"k: (-0.07699199999999999,0.733425,0.531662);f:...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,"p,o",0,0,0.806
a11,pfkswkfsococg,"p: (0.008034,0.957082,0.6890539999999999);p: (...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,"p,o",0,0,0.862
a13,wptgkfsoc,"w: (-0.196022,1.35572,0.466005);p: (0.008034,0...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",t,"p,o",0,0,0.806
...,...,...,...,...,...,...,...,...
h63,pkfsbcgo,"p: (3,8,2);k: (8,8,2);f: (6,8,2);s: (4,8,2);b:...","[5.5,6.5,2],[5.5,4,2],[5.5,4,2],[5.5,4,2],[5.5...",p,0,0,0,0.785
h64,pkfsocgb,"p: (5,8,2);k: (4,8,2);f: (2,8,2);s: (7,8,2);o:...","[5.5,6.5,2],[5.5,4,2],[5.5,4,2],[5.5,4,2],[5.5...",p,0,0,0,0.785
h65,pfskocgb,"p: (9,8,2);f: (6,8,2);s: (2,8,2);k: (8,8,2);o:...","[5.5,6.5,2],[5.5,4,2],[5.5,4,2],[5.5,4,2],[5.5...",p,0,0,0,0.785


In [22]:
c = [round(x,2) for x in np.arange(1.1,2.0,0.1)]
k_strong = [round(x,2) for x in np.arange(0.1,0.9,0.1)]
k_medium = [round(x + 0.1,2) for x in k_strong]
k_food = [round(x,2) for x in np.arange(1.1,2.0,0.1)]
dim = [[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']]

In [23]:
param_grid = [{'c': c, 'k_strong': k_strong, 'k_medium': k_medium, 'k_food': k_food, 'dim': dim}]

In [24]:
#parameter_grid = [x for x in ParameterGrid(param_grid) if x['k_strong'] < x['k_medium']]
parameter_grid = [x for x in ParameterGrid(param_grid) if x['k_strong'] == x['k_medium'] - 0.1]

In [78]:
grid = [line for line in parameter_grid]
len(grid)

2835

In [107]:
grid

[{'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.1, 'k_medium': 0.2, 'k_strong': 0.1},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.1, 'k_medium': 0.5, 'k_strong': 0.4},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.1, 'k_medium': 0.6, 'k_strong': 0.5},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.1, 'k_medium': 0.7, 'k_strong': 0.6},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.1, 'k_medium': 0.9, 'k_strong': 0.8},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.2, 'k_medium': 0.2, 'k_strong': 0.1},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.2, 'k_medium': 0.5, 'k_strong': 0.4},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.2, 'k_medium': 0.6, 'k_strong': 0.5},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.2, 'k_medium': 0.7, 'k_strong': 0.6},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.2, 'k_medium': 0.9, 'k_strong': 0.8},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.3, 'k_medium': 0.2, 'k_strong': 0.1},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.3, 'k_medium': 0.5, 'k_strong': 0.4},
 {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.3, 'k_mediu

In [91]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,]):
    ''' Predicts sequence based on required objects, object coordinates, start coordinates of subject,
        parameters (c+k) and dimensionality.
        Input: Objects, object coordinates, start coordinates, c, k, dimension
        Output: Sequence of objects as str
    '''
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [102]:
def get_median(df, row, parameters, n):
    ''' Returns median edit distance (Damerau-Levenshtein) for n trials of sequence prediction.
    '''  
    c = parameters['c']
    k_strong = parameters['k_strong']
    k_medium = parameters['k_medium']
    k_food = parameters['k_food']
    dim = parameters['dim']
        
    objects = list(df.at[row,'sequence'])
    strong_k = list(df.at[row,'strong_k'].split(','))
    medium_k = list(df.at[row,'mid_k'].split(','))
    food_k = list(df.at[row,'food_k'].split(','))
    coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in df.at[row,'coordinates'].split(';'))}
    start_coordinates = list(ast.literal_eval(df.at[row, 'start_coordinates']))
    sequence = str(df.at[row,'sequence'])
        
    k1 = {obj: k_strong if obj in strong_k else k_medium if obj in medium_k else k_food if obj in food_k else 1.0 for obj in objects}
    c1 = {obj: c if obj in df.at[row, 'containment'] else 1.0 for obj in objects}
    
    editdists = [1 - damerauLevenshtein(sequence,''.join(predict_sequence(objects,coordinates,start_coordinates,c1,k1,dim))) for x in range(0,n)]   
    median = np.median(editdists)
        
    params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_medium) + ',' + str(k_food) + '; ' + str(dim[1])
    
    return median, params, sequence

In [108]:
%time get_median(df, 'a1', {'c': 1.1, 'dim': [1, 'x'], 'k_food': 1.1, 'k_medium': 0.3, 'k_strong': 0.1}, 100)

CPU times: user 38 ms, sys: 2 µs, total: 38 ms
Wall time: 37 ms


(0.6666666666666667, 'c: 1.1; k: 0.1,0.3,1.1; x', 'pocgkr')

In [104]:
def fill_dataframe(data, params_grid, n):
    ''' Calculates average edit distance for all combinations of parameters (c, k, dimension).
        Input: Dataframe with objects, coordinates, start coordinates, object categories
        Output: Dataframe with edit distance results (col name: parameters used)
    '''
    results = pd.DataFrame(index=data.index)
    
    #for row in range(0,len(data)):
    for row in data.index:
        #print(row)
        for line in params_grid:
            median, params, sequence = get_median(data, row, line, n)
            results.at[row,params] = median
            results.at['mean',params] = results[params].mean()
        
        results.at[row,'error'] = data.at[row,'error']
        results.at[row, 'sequence'] = data.at[row, 'sequence']
                                
    return results

In [ ]:
#

In [105]:
%time results_test = fill_dataframe(df, parameter_grid, 100)

KeyboardInterrupt: 

In [106]:
results_test

,"c: 1.1; k: 0.1,0.2,1.1; x","c: 1.1; k: 0.4,0.5,1.1; x","c: 1.1; k: 0.5,0.6,1.1; x","c: 1.1; k: 0.6,0.7,1.1; x","c: 1.1; k: 0.8,0.9,1.1; x","c: 1.1; k: 0.1,0.2,1.2; x","c: 1.1; k: 0.4,0.5,1.2; x","c: 1.1; k: 0.5,0.6,1.2; x","c: 1.1; k: 0.6,0.7,1.2; x","c: 1.1; k: 0.8,0.9,1.2; x",...,"c: 1.9; k: 0.5,0.6,1.8; xyz","c: 1.9; k: 0.6,0.7,1.8; xyz","c: 1.9; k: 0.8,0.9,1.8; xyz","c: 1.9; k: 0.1,0.2,1.9; xyz","c: 1.9; k: 0.4,0.5,1.9; xyz","c: 1.9; k: 0.5,0.6,1.9; xyz","c: 1.9; k: 0.6,0.7,1.9; xyz","c: 1.9; k: 0.8,0.9,1.9; xyz",error,sequence
ID,,,,,,,,,,,,,,,,,,,,,
a1,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,...,0.333333,0.333333,0.333333,0.166667,0.166667,0.333333,0.333333,0.333333,0.723,pocgkr
a3,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,...,0.125000,0.125000,0.250000,0.125000,0.125000,0.125000,0.125000,0.250000,0.785,cgwpcfks
a5,0.555556,0.555556,0.555556,0.555556,0.555556,0.555556,0.555556,0.555556,0.555556,0.555556,...,0.444444,0.444444,0.666667,0.444444,0.444444,0.444444,0.444444,0.666667,0.806,kfsfkspwg
a11,0.461538,0.461538,0.461538,0.461538,0.461538,0.461538,0.461538,0.461538,0.461538,0.461538,...,0.384615,0.461538,0.461538,0.384615,0.384615,0.384615,0.461538,0.461538,0.862,pfkswkfsococg
a13,0.111111,0.111111,0.111111,0.111111,0.222222,0.111111,0.111111,0.111111,0.111111,0.222222,...,0.222222,0.000000,0.000000,0.333333,0.222222,0.222222,0.000000,0.000000,0.806,wptgkfsoc
a16,0.400000,0.400000,0.400000,0.400000,0.600000,0.400000,0.400000,0.400000,0.400000,0.600000,...,0.400000,0.400000,0.200000,0.600000,0.600000,0.400000,0.400000,0.200000,0.666,pwfkg
a19,0.166667,0.166667,0.333333,0.333333,0.333333,0.166667,0.166667,0.333333,0.333333,0.333333,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.723,powcer
a22,0.285714,0.285714,0.285714,0.285714,0.142857,0.285714,0.285714,0.285714,0.285714,0.142857,...,0.285714,0.142857,0.285714,0.428571,0.428571,0.285714,0.142857,0.285714,0.756,coprerk
a26,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.200000,...,0.400000,0.300000,0.200000,0.400000,0.400000,0.400000,0.300000,0.200000,0.824,wopofkkskf


In [79]:
def get_lowest_error(results):
    ''' Returns lowest error in dataframe, index of lowest error, mean of lowest error col and 
    the updated dataframe.

    Input: Dataframe of results
    '''

    for col in list(results):
        if col != 'sequence' and col != 'error' and col != 'ID':
            results.loc['mean', col] = results[col].mean()
    lowest = min(results.loc['mean'])
    mean = list(results.loc['mean'])

    return lowest, results.columns[(results.loc['mean'] == lowest)], mean, results

In [84]:
lowest, lowest_idx, mean, results_mean = get_lowest_error(results_test)
lowest, lowest_idx, np.nanmean(results_test['error'])

(0.2529454604297041,
 Index(['c: 1.4; k: 0.8,0.9,1.5; yz'], dtype='object'),
 0.7609583333333334)

In [88]:
results_test['c: 1.4; k: 0.8,0.9,1.5; yz']

ID
a1        0.166667
a3        0.250000
a5        0.555556
a11       0.384615
a13       0.111111
a16       0.200000
a19       0.166667
a22       0.142857
a26       0.100000
a29       0.333333
a31       0.200000
a35       0.250000
a37       0.076923
a39       0.166667
a40       0.000000
a43       0.166667
a45       0.363636
a47       0.666667
a51       0.071429
a55       0.066667
a56       0.090909
a59       0.125000
a60       0.066667
a61       0.235294
a69       0.125000
a73       0.250000
a76       0.000000
a77       0.444444
a80       0.285714
a85       0.200000
a87       0.125000
a89       0.153846
a93       0.333333
a99       0.300000
a100      0.285714
a102      0.300000
a106      0.000000
a108      0.500000
a109      0.000000
a114      0.333333
a116      0.333333
a118      0.200000
a120      0.100000
a128      0.000000
a131      0.200000
a137      0.000000
a142      0.400000
a149      0.111111
a152      0.125000
a153      0.125000
a156      0.000000
a158      0.600000
a161     